# Writing and Using a Bespoke Model Interface

In [1]:
import emat
import os
import pandas as pd
import numpy as np
import gzip
from emat.util.show_dir import show_dir, show_file_contents

This notebook is meant to illustrate the use of TMIP-EMAT's
various modes of operation.  It provides an illustration of how to use 
TMIP-EMAT and the demo interface to run the command line version
of the [Road Test](https://tmip-emat.github.io/source/emat.examples/RoadTest/road_test_yaml.html) 
model. A similar approach can be developed to run
any transportation model that can be run from the command line, including
for proprietary modeling tools that are typically run from a graphical
user interface (GUI) but that provide command line access also.

In this example notebook, we will activate some logging features.  The 
same logging utility is written directly into the EMAT and the
`core_files_demo.py` module. This will give us a view of what's happening
inside the code as it runs.

In [2]:
import logging
from emat.util.loggers import log_to_stderr
log = log_to_stderr(logging.INFO)

## Connecting to the Model

The interface for this model is located in the `core_files_demo.py`
module, which we will import into this notebook.  This file is extensively
documented in comments, and is a great starting point for new users
who want to write an interface for a new bespoke travel demand model.

In [3]:
import core_files_demo

Within this module, you will find a definition for the 
`RoadTestFileModel` class.

We initialize an instance of the model interface object.
If you look at the module code, you'll note the `__init__` function
does a number of things, including creating a temporary directory
to work in, copying the needed files into this temporary directory,
loading the scope, and creating a SQLite database to work within.
For your implementation, you might or might not do any of these steps.
In particular, you'll probably want to use a database that is
not in a temporary location, so that the results will be available
after this notebook is closed.

In [4]:
fx = core_files_demo.RoadTestFileModel()

[00:02.89] MainProcess/WARNING: changing cwd to /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmptek60l8k
[00:02.92] MainProcess/INFO: running script scope.sql
[00:02.93] MainProcess/INFO: running script exp_design.sql
[00:02.94] MainProcess/INFO: running script meta_model.sql


Once we have loaded the `RoadTestFileModel` class, we have
a number of files available in the "master_directory" that 
was created as that temporary directory:

In [5]:
show_dir(fx.master_directory.name)

tmptek60l8k/
├── road-test-demo.db
├── road-test-files/
│   ├── demo-inputs-l.yml
│   └── demo-inputs-x.yml.template
├── road-test-model-config.yml
└── road-test-scope.yml


## Understanding Directories

The TMIP-EMAT interface design for files-based bespoke models uses
pointers for several directories to control the operation of the 
model.

- **local_directory**  
    This is the working directory for this instance of TMIP-EMAT,
    not that for the core model itself. Typically it can be Python's 
    usual current working directory, accessible via `os.getcwd()`.
    In this directory typically you'll have a TMIP-EMAT model 
    configuration *yaml* file, a scope definition *yaml* file, and
    a sub-directory containing the files needed to run the core model
    itself.

- **model_path**  
    The relative path from the `local_directory` to the directory where
    the core model files are located.  When the core model itself is actually
    run, this should be to the "current working directory" for that run.
    The `model_path` must be given in the model config *yaml* file.

- **rel_output_path**  
    The relative path from the `model_path` to the directory where
    the core model output files are located. The default value of this 
    path is "./Outputs" but this can be overridden by setting 
    `rel_output_path` in the model config *yaml* file. If the outputs
    are comingled with other input files in the core model directory,
    this can be set to "." (just a dot).

- **archive_path**  
    The path where model archive directories can be found. This path
    must be given in the model config *yaml* file. It can be given as
    an absolute path, or a relative path. If it is a relative path, 
    it should be relative to the `local_directory`.
    
These directories, especially the ones other than the `local_directory`,
are defined in a model configuration *yaml* file. This makes it easy to
change the directory pointers when moving TMIP-EMAT between different
machines that may have different file system structures.

## Single Run Operation for Development and Debugging

Before we take on the task of running this model in exploratory mode, we'll
want to make sure that our interface code is working correctly. To check each
of the components of the interface (setup, run, post-process, load-measures,
and archive), we can run each individually in sequence, and inspect the results
to make sure they are correct.

### setup

This method is the place where the core model *set up* takes place,
including creating or modifying files as necessary to prepare
for a core model run.  When running experiments, this method
is called once for each core model experiment, where each experiment
is defined by a set of particular values for both the exogenous
uncertainties and the policy levers.  These values are passed to
the experiment only here, and not in the `run` method itself.
This facilitates debugging, as the `setup` method can be used 
without the `run` method, as we do here. This allows us to manually
inspect the prepared files and ensure they are correct before
actually running a potentially expensive model.

Each input exogenous uncertainty or policy lever can potentially
be used to manipulate multiple different aspects of the underlying
core model.  For example, a policy lever that includes a number of
discrete future network "build" options might trigger the replacement
of multiple related network definition files.  Or, a single uncertainty
relating to the cost of fuel might scale both a parameter linked to
the modeled per-mile cost of operating an automobile and the
modeled total cost of fuel used by transit services.

For this demo model, running the core model itself in files mode 
requires two configuration files to be available, one for levers and
another for uncertainties.  These two files are provided in the demo
in two ways: as a runnable base file (for the levers) and as a template
file (for the uncertainties).

The levers file is a *ready-to-use* file (for this demo, in YAML format,
although your model may use a different file format for input files).
It has default values pre-coded into the file, and to modify this 
file for use by EMAT the `setup` method needs to parse and edit this
file to swap out the default values for new ones in each experiment.
This can be done using regular expressions (as in this demo), or any other method you
like to edit the file appropriately.  The advantage of this approach
is that the base file is ready to use with the core model as-is, facilitating
the use of this file outside the EMAT context.

In [6]:
show_file_contents(fx.master_directory.name, 'road-test-files', 'demo-inputs-l.yml')

---
# This file defines lever values for the files-based
# Road Test example.  It is intentionally a complex way
# to implement this Python-based model, designed to
# demonstrate how to use a files-based model called
# from the command line.
expand_capacity: 10
amortization_period: 30
interest_rate_lock: False
debt_type: GO Bond
lane_width: 10
mandatory_unused_lever: 42
...


By contrast, the uncertainties file is in a *template* format. The
values of the parameters that will be manipulated by EMAT for each 
experiment are not given by default values, but instead 
each value to be set is indicated in the file by a unique token that is easy to
search and replace, and definitely not something that appear in any script otherwise.
This approach makes the text-substitution code that is used in this module much
simpler and less prone to bugs.  But there is a small downside of this approach:
every parameter must definitely be replaced in this process, as the template file
is unusable outside the EMAT context, and also every unique token needs to be replaced. 

In [7]:
show_file_contents(fx.master_directory.name, 'road-test-files', 'demo-inputs-x.yml.template')

---
# This file defines uncertainty values for the files-based
# Road Test example.  It is intentionally a complex way
# to implement this Python-based model, designed to
# demonstrate how to use a files-based model called
# from the command line.
alpha: __EMAT_PROVIDES_VALUE__ALPHA__
beta: __EMAT_PROVIDES_VALUE__BETA__
input_flow: __EMAT_PROVIDES_VALUE__INPUT_FLOW__
value_of_time: __EMAT_PROVIDES_VALUE__VALUE_OF_TIME__
labor_unit_cost_expansion: __EMAT_PROVIDES_VALUE__LABOR_UNIT_COST_EXPANSION__
materials_unit_cost_expansion: __EMAT_PROVIDES_VALUE__MATERIALS_UNIT_COST_EXPANSION__
interest_rate: __EMAT_PROVIDES_VALUE__INTEREST_RATE__
yield_curve: __EMAT_PROVIDES_VALUE__YIELD_CURVE__
...


Regardless of which file management system you use, the `setup` method
is the place to make edits to these input files and write them into 
your working directory.  To do so,
the `setup` method takes one argument: a dictionary containing key-value
pairs that assign a particular value to each input (exogenous uncertainty 
or policy lever) that is defined in the model scope.  The keys must match 
exactly with the names of the parameters given in the scope. 

If you have written your `setup` method to call the super-class `setup`,
you will find that if you give keys as input that are not defined in
the scope, you'll get a KeyError.

In [8]:
bad_params = {
    'name_not_in_scope': 'is_a_problem',
}

try:
    fx.setup(bad_params)
except KeyError as error:
    log.error(repr(error))

[00:02.97] MainProcess/INFO: RoadTestFileModel SETUP...
[00:02.97] MainProcess/ERROR: KeyError("'name_not_in_scope' not found in scope parameters")


On the other hand, your custom model may or may not allow you to leave out
some parameters.  It is up to you to decide how to handle missing values, 
either by setting them at their default values or raising an error. In 
normal operation, parameters typically won't be left out from the design
of experiments, so it is not usually important to monitor this carefully.

In our example module's `setup`, all of the uncertainty values must be given,
because the template file would be unusable otherwise. But the policy levers 
can be omitted, and if so they are left at their default values in the 
original file.  Note that the default values in that file are not strictly
consistent with the default values in the scope file, and TMIP-EMAT does 
nothing on its own to address this discrepancy.

In [9]:
params = {
    'expand_capacity': 75,
    'amortization_period': 25,
    'debt_type': "Paygo",
    'alpha': 0.1234,
    'beta': 4.0,
    'input_flow': 100,
    'value_of_time': 0.075,
    'unit_cost_expansion': 100,
    'interest_rate': 0.035,
    'yield_curve': 0.01,
} # interest_rate_lock is missing, that's ok

fx.setup(params)

[00:02.97] MainProcess/INFO: RoadTestFileModel SETUP...
[00:02.98] MainProcess/INFO: RoadTestFileModel SETUP complete


After running `setup` successfully, we will have overwritten the 
"demo-inputs-l.yml" file with new values, and written a new 
"demo-inputs-x.yml" file into the model working directory with those
values.

In [10]:
show_dir(fx.local_directory)

tmptek60l8k/
├── road-test-demo.db
├── road-test-files/
│   ├── demo-inputs-l.yml
│   ├── demo-inputs-x.yml
│   └── demo-inputs-x.yml.template
├── road-test-model-config.yml
└── road-test-scope.yml


In [11]:
show_file_contents(fx.local_directory, 'road-test-files', 'demo-inputs-l.yml')

---
# This file defines lever values for the files-based
# Road Test example.  It is intentionally a complex way
# to implement this Python-based model, designed to
# demonstrate how to use a files-based model called
# from the command line.
expand_capacity: 75
amortization_period: 25
interest_rate_lock: False
debt_type: Paygo  
lane_width: 10
mandatory_unused_lever: 42
...


In [12]:
show_file_contents(fx.local_directory, 'road-test-files', 'demo-inputs-x.yml')

---
# This file defines uncertainty values for the files-based
# Road Test example.  It is intentionally a complex way
# to implement this Python-based model, designed to
# demonstrate how to use a files-based model called
# from the command line.
alpha: 0.1234
beta: 4.0
input_flow: 100
value_of_time: 0.075
labor_unit_cost_expansion: 60.0
materials_unit_cost_expansion: 40.0
interest_rate: 0.035
yield_curve: 0.01
...


### run

The `run` method is the place where the core model run takes place.
Note that this method takes no arguments; all the input
exogenous uncertainties and policy levers are delivered to the
core model in the `setup` method, which will be executed prior
to calling this method. This facilitates debugging, as the `setup`
method can be used without the `run` method as we did above, allowing
us to manually inspect the prepared files and ensure they
are correct before actually running a potentially expensive model.

In [13]:
fx.run()

[00:03.00] MainProcess/INFO: RoadTestFileModel RUN ...
[00:05.99] MainProcess/INFO: RoadTestFileModel RUN complete


The `RoadTestFileModel` class includes a custom `last_run_logs` method,
which displays both the "stdout" and "stderr" logs generated by the 
model executable during the most recent call to the `run` method.
We can use this method for debugging purposes, to identify why the 
core model crashes (if it does crash).  In this first test it did not
crash, and the logs look good.

In [14]:
fx.last_run_logs()

=== STDOUT ===
[2020-07-01 13:41:02,612] emat.RoadTest.INFO: running emat-road-test-demo
[2020-07-01 13:41:02,618] emat.RoadTest.INFO: emat-road-test-demo completed without errors

=== END OF LOG ===


### post-process

There is an (optional) `post_process` step that is separate from the `run` step.

Post-processing differs from the main model run in two important ways:

- It can be run to efficiently generate a subset of performance measures.
- It can be run based on archived model main-run core model results.

Both features are designed to support workflows where new performance 
measures are added to the exploratory scope after the main model run(s)
are completed. By allowing the `post_process` method to be run only for a 
subset of measures, we can avoid replicating possibly expensive 
post-processing steps when we have already completed them, or when they
are not needed for a particular application.  

For example, consider an exploratory modeling activity where the scope 
at the time of the initial model run experiments was focused on highway
measures, and transit usage was not explored extensively, and no 
network assignment was done for transit trips when the experiments were
initially run.  By creating a post-process step to run the transit 
network assignment, we can apply that step to existing archived results,
as well as have it run automatically for future model experients
where transit usage is under study, but continue to omit it for future 
model experients where we do not need it.

An optional `measure_names` argument allows the post-processor to
identify which measures need additional computational effort to generate,
and to skip excluded measures that are not currently of interest, or
which have already been computed and do not need to be computed again.

The post processing is isolated from the main model run to allow it to
be run later using archived model results.  When executed directly 
after a core model run, it will operate on the results of the model
stored in the local working directory.  However, it can also be
used with an optional `output_path` argument, which can be pointed at
a model archive directory instead of the local working directory.

A consequence of this (and an intentional limitation) is that the 
`post_process` method should only use files from the set of files 
that are or will be archived from the core model run, and not attempt
to use other non-persistent temporary or intermediate files that 
will not be archived.

In [15]:
fx.post_process()

[00:06.00] MainProcess/INFO: RoadTestFileModel POST-PROCESS ...
[00:06.03] MainProcess/INFO: RoadTestFileModel POST-PROCESS complete


At this point, the model's output performance measures should be available in one
or more output files that can be read in the next step.  For this example, the
results are written to two separate files: 'output_1.csv.gz' and 'output.yaml'.

In [16]:
show_file_contents(fx.local_directory, 'road-test-files', "Outputs", "output.yaml")

build_travel_time: 60.78943107038733
no_build_travel_time: 67.404
time_savings: 6.614568929612666



Note in this example, some of the values in the `output_1.csv.gz` file
are intentionally manipulated in a contrived manner, so that there is 
some work for the post-processor to do.

In [17]:
show_file_contents(fx.local_directory, 'road-test-files', "Outputs", "output_1.csv.gz")

,value_of_time_savings,present_cost_expansion,cost_of_capacity_expansion,net_benefits
exp,1.0508604102769246,,1.3651577800056909,
plain,49.60926697209499,7500.0,311.2700117047018,-261.6607447326066



### load-measures

The `load_measures` method is the place to actually reach into
files in the core model's run results and extract performance
measures, returning a dictionary of key-value pairs for the 
various performance measures. It takes an optional list giving a 
subset of performance measures to load, and like the `post_process` 
method also can be pointed at an archive location instead of loading 
measures from the local working directory (which is the default).
The `load_measures` method should not do any post-processing
of results (i.e. it should read from but not write to the model
outputs directory).

In [18]:
fx.load_measures()

{'value_of_time_savings': 49.60926697209499,
 'present_cost_expansion': 7500.0,
 'cost_of_capacity_expansion': 311.2700117047018,
 'net_benefits': -261.6607447326066,
 'build_travel_time': 60.78943107038733,
 'no_build_travel_time': 67.404,
 'time_savings': 6.614568929612666}

You may note that the implementation of `RoadTestFileModel` in the `core_files_demo` module
does not actually include a `load_measures` method itself, but instead inherits this method
from the `FilesCoreModel` superclass. The instructions on how to actually find the relevant
performance measures for this file are instead loaded into table parsers, which are defined
in the `RoadTestFileModel.__init__` constructor.  There are [details and illustrations
of how to write and use parsers in the file parsing examples page of the TMIP-EMAT documentation.](https://tmip-emat.github.io/source/emat.models/table_parse_example.html)

### archive

The `archive` method copies the relevant model output files to an archive location for 
longer term storage.  The particular archive location is based on the experiment id
for a particular experiment, and can be customized if desired by overloading the 
`get_experiment_archive_path` method.  This customization is not done in this demo,
so the default location is used.

In [19]:
fx.get_experiment_archive_path(parameters=params)

'/var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmptek60l8k/./archive/scp_EMAT Road Test/exp_1'

Actually running the `archive` method should copy any relevant output files
from the `model_path` of the current active model into a subdirectory of `archive_path`.

In [20]:
fx.archive(params)

[00:06.08] MainProcess/INFO: RoadTestFileModel ARCHIVE
 from: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmptek60l8k/road-test-files
   to: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmptek60l8k/./archive/scp_EMAT Road Test/exp_1


In [21]:
show_dir(fx.local_directory)

tmptek60l8k/
├── archive/
│   └── scp_EMAT Road Test/
│       └── exp_1/
│           ├── demo-inputs-l.yml
│           ├── demo-inputs-x.yml
│           ├── demo-inputs-x.yml.template
│           ├── emat-road-test.log
│           ├── output.csv.gz
│           ├── output.yaml
│           └── Outputs/
│               ├── output.yaml
│               └── output_1.csv.gz
├── road-test-demo.db
├── road-test-files/
│   ├── demo-inputs-l.yml
│   ├── demo-inputs-x.yml
│   ├── demo-inputs-x.yml.template
│   ├── emat-road-test.log
│   ├── output.csv.gz
│   ├── output.yaml
│   └── Outputs/
│       ├── output.yaml
│       └── output_1.csv.gz
├── road-test-model-config.yml
└── road-test-scope.yml


It is permissible, but not required, to simply copy the entire contents of the 
former to the latter, as is done in this example. However, if the current active model
directory has a lot of boilerplate files that don't change with the inputs, or
if it becomes full of intermediate or temporary files that definitely will never
be used to compute performance measures, it can be advisable to selectively copy
only relevant files. In that case, those files and whatever related sub-directory
tree structure exists in the current active model should be replicated within the
experiments archive directory.

## Normal Operation for Running Multiple Experiments

For this demo, we'll create a design of experiments with only 8 experiments.
The `design_experiments` method of the `RoadTestFileModel` object is not defined
in the custom `core_files_demo` written for this model, but rather is a generic
function provide by the TMIP-EMAT main library.
Real applications will typically use a larger number of experiments, but this small number
is sufficient to demonstrate the operation of the tools.

In [22]:
design1 = fx.design_experiments(design_name='lhs_1', n_samples=8)
design1

alpha  amortization_period      beta debt_type  \
experiment                                                      
2           0.134750                   36  4.642370  Rev Bond   
3           0.115907                   50  5.242315  Rev Bond   
4           0.178456                   30  3.510139     Paygo   
5           0.110023                   44  4.887030     Paygo   
6           0.161977                   24  3.865644   GO Bond   
7           0.173449                   21  4.094118     Paygo   
8           0.141973                   19  5.331978   GO Bond   
9           0.193762                   40  4.453382  Rev Bond   

            expand_capacity  input_flow  interest_rate  interest_rate_lock  \
experiment                                                                   
2                 24.901018          85       0.039219               False   
3                 11.985022         114       0.025997                True   
4                 72.399552         121       0.029066                True   
5                 28.565637         127       0.034673                True   
6                 82.811459         135       0.028634                True   
7                 43.476172         142       0.033847               False   
8                 52.445940          98       0.031519               False   
9                 92.278968          95       0.037907               False   

            unit_cost_expansion  value_of_time  yield_curve  free_flow_time  \
experiment                                                                    
2                    143.519760       0.017005     0.003280              60   
3                    107.836739       0.145378     0.009659              60   
4                    127.838010       0.067821    -0.000839              60   
5                    125.799829       0.083593     0.002714              60   
6                    118.989447       0.054968     0.006745              60   
7                    133.480778       0.184904     0.017674              60   
8                     99.317350       0.115228     0.014752              60   
9                    103.543415       0.092517     0.014360              60   

            initial_capacity  
experiment                    
2                        100  
3                        100  
4                        100  
5                        100  
6                        100  
7                        100  
8                        100  
9                        100

The `run_experiments` command will automatically run the model once for each experiment in the named design.
The demo command line version of the road test model is (intentionally) a little bit slow, so will take a few
seconds to conduct these eight model experiment runs.

In [23]:
fx.run_experiments(design_name='lhs_1')

[00:06.15] MainProcess/INFO: performing 8 scenarios/policies * 1 model(s) = 8 experiments
[00:06.16] MainProcess/INFO: performing experiments sequentially
[00:06.16] MainProcess/INFO: RoadTestFileModel SETUP...
[00:06.17] MainProcess/INFO: RoadTestFileModel SETUP complete
[00:06.17] MainProcess/INFO: RoadTestFileModel RUN ...
[00:09.27] MainProcess/INFO: RoadTestFileModel RUN complete
[00:09.27] MainProcess/INFO: RoadTestFileModel POST-PROCESS ...
[00:09.29] MainProcess/INFO: RoadTestFileModel POST-PROCESS complete
[00:09.29] MainProcess/INFO: RoadTestFileModel ARCHIVE
 from: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmptek60l8k/road-test-files
   to: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmptek60l8k/./archive/scp_EMAT Road Test/exp_2
[00:09.30] MainProcess/INFO: 1 cases completed
[00:09.31] MainProcess/INFO: RoadTestFileModel SETUP...
[00:09.32] MainProcess/INFO: RoadTestFileModel SETUP complete
[00:09.32] MainProcess/INFO: RoadTestFileModel RUN ...
[00:12.44] MainPr

alpha      beta  input_flow  value_of_time  \
experiment                                                  
2           0.134750  4.642370          85       0.017005   
3           0.115907  5.242315         114       0.145378   
4           0.178456  3.510139         121       0.067821   
5           0.110023  4.887030         127       0.083593   
6           0.161977  3.865644         135       0.054968   
7           0.173449  4.094118         142       0.184904   
8           0.141973  5.331978          98       0.115228   
9           0.193762  4.453382          95       0.092517   

            unit_cost_expansion  interest_rate  yield_curve  expand_capacity  \
experiment                                                                     
2                    143.519760       0.039219     0.003280        24.901018   
3                    107.836739       0.025997     0.009659        11.985022   
4                    127.838010       0.029066    -0.000839        72.399552   
5                    125.799829       0.034673     0.002714        28.565637   
6                    118.989447       0.028634     0.006745        82.811459   
7                    133.480778       0.033847     0.017674        43.476172   
8                     99.317350       0.031519     0.014752        52.445940   
9                    103.543415       0.037907     0.014360        92.278968   

            amortization_period debt_type  interest_rate_lock  free_flow_time  \
experiment                                                                      
2                            36  Rev Bond               False              60   
3                            50  Rev Bond                True              60   
4                            30     Paygo                True              60   
5                            44     Paygo                True              60   
6                            24   GO Bond                True              60   
7                            21     Paygo               False              60   
8                            19   GO Bond               False              60   
9                            40  Rev Bond               False              60   

            initial_capacity  no_build_travel_time  build_travel_time  \
experiment                                                              
2                        100             63.802031          61.354318   
3                        100             73.822149          67.635961   
4                        100             80.905959          63.090263   
5                        100             81.228935          66.217725   
6                        100             91.004725          63.010343   
7                        100            103.733064          69.975507   
8                        100             67.648456          60.807614   
9                        100                   NaN                NaN   

            time_savings  value_of_time_savings  net_benefits  \
experiment                                                      
2               2.447713               3.537924   -175.134011   
3               6.186188             102.524606     42.361931   
4              17.815696             146.201784   -183.229667   
5              15.011210             159.363387     61.567385   
6              27.994382             207.737074   -374.545620   
7              33.757557             886.351136    604.765873   
8               6.840842              77.249238   -282.042005   
9                    NaN                    NaN           NaN   

            cost_of_capacity_expansion  present_cost_expansion  
experiment                                                      
2                           178.671935             3573.788162  
3                            60.162675             1292.425695  
4                           329.431451             9255.414627  
5                            97.796001             3593.552248  
6                   

### Re-running Failed Experiments

If you pay attention to the logged output, you might notice that one of the 
experiments (the last one) failed.  We can see `NaN` values in the outputs.

In [24]:
results = fx.read_experiment_measures('lhs_1')
results

no_build_travel_time  build_travel_time  time_savings  \
experiment                                                          
2                      63.802031          61.354318      2.447713   
3                      73.822149          67.635961      6.186188   
4                      80.905959          63.090263     17.815696   
5                      81.228935          66.217725     15.011210   
6                      91.004725          63.010343     27.994382   
7                     103.733064          69.975507     33.757557   
8                      67.648456          60.807614      6.840842   
9                            NaN                NaN           NaN   

            value_of_time_savings  net_benefits  cost_of_capacity_expansion  \
experiment                                                                    
2                        3.537924   -175.134011                  178.671935   
3                      102.524606     42.361931                   60.162675   
4                      146.201784   -183.229667                  329.431451   
5                      159.363387     61.567385                   97.796001   
6                      207.737074   -374.545620                  582.282695   
7                      886.351136    604.765873                  281.585263   
8                       77.249238   -282.042005                  359.291243   
9                             NaN           NaN                         NaN   

            present_cost_expansion  
experiment                          
2                      3573.788162  
3                      1292.425695  
4                      9255.414627  
5                      3593.552248  
6                      9853.689681  
7                      5803.233337  
8                      5208.791736  
9                              NaN

We can collect the id's of the failed experiments programmatically. To collect all the experiments that are
missing any performance measure output, we can do this:

In [25]:
fails = results.isna().any(axis=1)
failed_experiment_ids = fails.index[fails]
failed_experiment_ids

Int64Index([9], dtype='int64', name='experiment')

When there is an error (thrown as a `subprocess.CalledProcessError`)
during the execution of a `FilesCoreModel`, the output from stdout
and stderr are written to log files in the archive location, instead
of having the legit model outputs written there.

We can see the log output by reading in the log file, like this:

In [26]:
error_log = os.path.join(
    fx.get_experiment_archive_path(9), 
    'error.stdout.log'
)
with open(error_log, 'r') as stdout:
    error_log_content = stdout.read()
    
print(error_log_content)

[2020-07-01 13:41:25,675] emat.RoadTest.INFO: running emat-road-test-demo
[2020-07-01 13:41:25,678] emat.RoadTest.ERROR: Random crash, ha ha!



Here we see the log file is explicitly taunting us about 
randomly crashing the model run.  That's fine -- we wanted to
crash the execution randomly to show what to do in this event, cause it happens 
sometimes.  Maybe a disk filled up, or there is an intermittent
license problem that causes a failure one in a while.  If that's the
case and we can fix it just by re-running, awesome!

We can load just the failed experiments to try them again.

In [27]:
failed_experiments = fx.read_experiment_parameters(experiment_ids=failed_experiment_ids)
failed_experiments

free_flow_time  initial_capacity     alpha      beta  input_flow  \
experiment                                                                     
9                       60               100  0.193762  4.453382          95   

            value_of_time  unit_cost_expansion  interest_rate  yield_curve  \
experiment                                                                   
9                0.092517           103.543415       0.037907      0.01436   

            expand_capacity  amortization_period debt_type  interest_rate_lock  
experiment                                                                      
9                 92.278968                   40  Rev Bond               False

Normally, there is a "short circuit" process that will
prevent re-running a core model experiment, instead the performance measure results
will simply be loaded from the database, which is typically much faster than
actually running the core model.  But, if the performance measures stored in the
database are junk, we will not want to trigger the short circuit system, and
actually run the full core model again.  To do so, we can disable the
short circuit like this:

In [28]:
fx.allow_short_circuit = False

Now we can re-run the failed experiment.  If it failed because of a transient error, 
e.g. a disk space problem that's been fixed, then perhaps we can simply re-run the model
and it will work.

In [29]:
fx.run_experiments(failed_experiments)

[00:29.13] MainProcess/INFO: performing 1 scenarios/policies * 1 model(s) = 1 experiments
[00:29.15] MainProcess/INFO: performing experiments sequentially
[00:29.15] MainProcess/INFO: RoadTestFileModel SETUP...
[00:29.15] MainProcess/INFO: RoadTestFileModel SETUP complete
[00:29.15] MainProcess/INFO: RoadTestFileModel RUN ...
[00:31.72] MainProcess/INFO: RoadTestFileModel RUN complete
[00:31.72] MainProcess/INFO: RoadTestFileModel POST-PROCESS ...
[00:31.73] MainProcess/INFO: RoadTestFileModel POST-PROCESS complete
[00:31.74] MainProcess/INFO: RoadTestFileModel ARCHIVE
 from: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmptek60l8k/road-test-files
   to: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmptek60l8k/./archive/scp_EMAT Road Test/exp_9
[00:31.75] MainProcess/INFO: 1 cases completed
[00:31.75] MainProcess/INFO: experiments finished


alpha      beta  input_flow  value_of_time  \
experiment                                                  
9           0.193762  4.453382          95       0.092517   

            unit_cost_expansion  interest_rate  yield_curve  expand_capacity  \
experiment                                                                     
9                    103.543415       0.037907      0.01436        92.278968   

            amortization_period debt_type  interest_rate_lock  free_flow_time  \
experiment                                                                      
9                            40  Rev Bond               False              60   

            initial_capacity  no_build_travel_time  build_travel_time  \
experiment                                                              
9                        100             69.251572          60.503221   

            time_savings  value_of_time_savings  net_benefits  \
experiment                                                      
9               8.748351              76.890662   -385.524839   

            cost_of_capacity_expansion  present_cost_expansion  
experiment                                                      
9                             462.4155              9554.87952

Much better!  Now we can see we have a more complete set of outputs, without the NaN's.  Hooray!

In [30]:
results = fx.db.read_experiment_all(scope_name=fx.scope.name, design_name='lhs_1')
results

free_flow_time initial_capacity     alpha     beta input_flow  \
experiment                                                                 
2                      60              100   0.13475  4.64237         85   
3                      60              100  0.115907  5.24231        114   
4                      60              100  0.178456  3.51014        121   
5                      60              100  0.110023  4.88703        127   
6                      60              100  0.161977  3.86564        135   
7                      60              100  0.173449  4.09412        142   
8                      60              100  0.141973  5.33198         98   
9                      60              100  0.193762  4.45338         95   

           value_of_time unit_cost_expansion interest_rate yield_curve  \
experiment                                                               
2              0.0170047              143.52     0.0392193  0.00327971   
3               0.145378             107.837     0.0259967  0.00965899   
4               0.067821             127.838     0.0290662 -0.00083927   
5              0.0835929               125.8     0.0346728  0.00271409   
6              0.0549679             118.989     0.0286337  0.00674486   
7               0.184904             133.481     0.0338467   0.0176743   
8               0.115228             99.3173     0.0315185   0.0147519   
9              0.0925175             103.543     0.0379069     0.01436   

           expand_capacity amortization_period debt_type interest_rate_lock  \
experiment                                                                    
2                   24.901                  36  Rev Bond                  0   
3                   11.985                  50  Rev Bond                  1   
4                  72.3996                  30     Paygo                  1   
5                  28.5656                  44     Paygo                  1   
6                  82.8115                  24   GO Bond                  1   
7                  43.4762                  21     Paygo                  0   
8                  52.4459                  19   GO Bond                  0   
9                   92.279                  40  Rev Bond                  0   

           no_build_travel_time build_travel_time time_savings  \
experiment                                                       
2                        63.802           61.3543      2.44771   
3                       73.8221            67.636      6.18619   
4                        80.906           63.0903      17.8157   
5                       81.2289           66.2177      15.0112   
6                       91.0047           63.0103      27.9944   
7                       103.733           69.9755      33.7576   
8                       67.6485           60.8076      6.84084   
9                       69.2516           60.5032      8.74835   

           value_of_time_savings net_benefits cost_of_capacity_expansion  \
experiment                                                                 
2                        3.53792     -175.134                    178.672   
3                        102.525      42.3619                    60.1627   
4                        146.202      -183.23                    329.431   
5                        159.363      61.5674                     97.796   
6                        207.737     -374.546                    582.283   
7                        886.351      604.766                    281.585   
8                        77.2492     -282.042                    359.291   
9                        76.8907     -385.525                    462.416   

           present_cost_expansion  
experiment                         
2                         3573.79  
3                         1292.43  
4                         9255.41  
5                         3593.55  
6                         9853.69  
7                         5803.23  
8       

## Multiprocessing for Running Multiple Experiments

The examples above are all single-process demonstrations of using TMIP-EMAT to run core model
experiments.  If your core model itself is multi-threaded or otherwise is designed to make 
full use of your multi-core CPU, or if a single core model run will otherwise max out some
computational resource (e.g. RAM, disk space) then single process operation should be sufficient.

If, on the other hand, your core model is such that you can run multiple independent instances of
the model side-by-side on the same machine, then you could benefit from a multiprocessing 
approach.  This can be accomplished by splitting a design of experiments over several
processes that you start manually, or by using an automatic multiprocessing library such as 
`dask.distributed`.

### Running a Subset of Experiments Manually

Suppose, for example, you wanted to distribute the workload of running experiments over several processes,
or even over several computers. If each process has file system access to the same TMIP-EMAT database of
experiments, we can orchestrate these experiments in parallel by manually splitting up the processes.

To begin with, we'll have one process create a complete design of experiments, and save it to the 
database (which happens automatically here).

In [31]:
design2 = fx.design_experiments(design_name='lhs_2', n_samples=8, random_seed=42)

Then, we can create set up a copy of the same model in a different process, even on a different
machine, as long as we point back to the same original database file. This implies the different
process has access to the file system where the original file is stored. It is valuable to
read and write to the same database file, not just a copy of the file, as this will obviate the need
to sync the experimental data manually afterwards.  In this demo, we'll 
just create a new directory to work in, but we'll point to the database in the original directory.
Instead of allowing our model to implicitly create a new database file in the new directory, we'll
instantiate a SQLiteDB object pointing to the original database.

In [32]:
database_filename = fx.db.database_path
db2 = emat.SQLiteDB(database_filename)

Now, `db2` is a `emat.SQLiteDB` object, which wraps a *new* connection to the *original* database.
Then, we'll pass that `db2` explicitly to the new `RoadTestFileModel` constructor, which will
create a complete copy of our model (other than the database) in a new directory.

In [33]:
fx2 = core_files_demo.RoadTestFileModel(db=db2)

[00:31.83] MainProcess/WARNING: changing cwd to /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpm4ri_fl2


To run a particular slice of a design of experiments, we need to load the experimental design first, 
and then pass that slice to the `run_experiments` function, instead of just giving the `design_name`.

In [34]:
design2 = fx.read_experiment_parameters('lhs_2')

For splitting the work across a number of similarly capable processes or machines,
the double-colon slice is convenient.  If, for example, you are splitting the work
over 4 computers, you can run each with slices `0::4`, `1::4`, `2::4`, and `3::4`.
This slices in skip-step manner, so slice below will run every 4th experiment
from the design, starting with experiment index 0 (i.e. the first one).  

In [35]:
fx2.run_experiments(design2.iloc[0::4])

[00:31.89] MainProcess/INFO: performing 2 scenarios/policies * 1 model(s) = 2 experiments
[00:31.91] MainProcess/INFO: performing experiments sequentially
[00:31.91] MainProcess/INFO: RoadTestFileModel SETUP...
[00:31.91] MainProcess/INFO: RoadTestFileModel SETUP complete
[00:31.91] MainProcess/INFO: RoadTestFileModel RUN ...
[00:34.50] MainProcess/INFO: RoadTestFileModel RUN complete
[00:34.50] MainProcess/INFO: RoadTestFileModel POST-PROCESS ...
[00:34.51] MainProcess/INFO: RoadTestFileModel POST-PROCESS complete
[00:34.52] MainProcess/INFO: RoadTestFileModel ARCHIVE
 from: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpm4ri_fl2/road-test-files
   to: /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpm4ri_fl2/./archive/scp_EMAT Road Test/exp_10
[00:34.52] MainProcess/INFO: 1 cases completed
[00:34.53] MainProcess/INFO: RoadTestFileModel SETUP...
[00:34.54] MainProcess/INFO: RoadTestFileModel SETUP complete
[00:34.54] MainProcess/INFO: RoadTestFileModel RUN ...
[00:37.17] MainP

alpha      beta  input_flow  value_of_time  \
experiment                                                  
10          0.114450  3.648104         100       0.034746   
14          0.132514  4.016263         133       0.107612   

            unit_cost_expansion  interest_rate  yield_curve  expand_capacity  \
experiment                                                                     
10                   120.196432       0.028321     0.001514         5.621927   
14                   131.922290       0.035993     0.015811        54.081760   

            amortization_period debt_type  interest_rate_lock  free_flow_time  \
experiment                                                                      
10                           34     Paygo                True              60   
14                           22     Paygo               False              60   

            initial_capacity  no_build_travel_time  build_travel_time  \
experiment                                                              
10                       100             66.867014          65.624845   
14                       100             84.993873          64.403269   

            time_savings  value_of_time_savings  net_benefits  \
experiment                                                      
10              1.242169               4.316063    -17.502784   
14             20.590604             294.700221    -37.110353   

            cost_of_capacity_expansion  present_cost_expansion  
experiment                                                      
10                           21.818848              675.735529  
14                          331.810575             7134.589600

Because we have linked the second model instance back to the same database, after
these experiments have finished we can access the results from the original `fx`
instance.

In [36]:
fx.read_experiment_measures('lhs_2')

no_build_travel_time  build_travel_time  time_savings  \
experiment                                                          
10                     66.867014          65.624845      1.242169   
14                     84.993873          64.403269     20.590604   

            value_of_time_savings  net_benefits  cost_of_capacity_expansion  \
experiment                                                                    
10                       4.316063    -17.502784                   21.818848   
14                     294.700221    -37.110353                  331.810575   

            present_cost_expansion  
experiment                          
10                      675.735529  
14                     7134.589600

It is important to note that for this manual multiprocessing technique to work, where
different processes run the model simultaneously, each process must be in a seperate 
Python instance (e.g. in seperate Jupyter notebooks, not in the same notebook as shown here).

### Automatic Multiprocessing for Running Multiple Experiments

The examples above are all essentially single-process demonstrations of using TMIP-EMAT to run core model
experiments, either by running all in one single process, or by having a user manually instantiate a number 
of single processes.  If your core model itself is multi-threaded or otherwise is designed to make 
full use of your multi-core CPU, or if a single core model run will otherwise max out some
computational resource (e.g. RAM, disk space) then single process operation should be sufficient.

If, on the other hand, your model is such that you can run multiple independent instances of
the model side-by-side on the same machine, but you don't want to manage the process of manually, 
then you could benefit from a multiprocessing approach that uses the `dask.distributed` library.  To
demonstrate this, we'll create yet another small design of experiments to run.

In [37]:
design3 = fx.design_experiments(design_name='lhs_3', n_samples=8, random_seed=3)
design3

alpha  amortization_period      beta debt_type  \
experiment                                                      
18          0.169616                   35  4.064396     Paygo   
19          0.192963                   46  4.973266   GO Bond   
20          0.150234                   31  5.429606  Rev Bond   
21          0.175508                   38  4.505154  Rev Bond   
22          0.140599                   21  5.035679   GO Bond   
23          0.113669                   25  3.831327     Paygo   
24          0.108685                   18  4.452232   GO Bond   
25          0.126817                   43  3.599748  Rev Bond   

            expand_capacity  input_flow  interest_rate  interest_rate_lock  \
experiment                                                                   
18                26.090367         101       0.034895                True   
19                56.044819         107       0.027205               False   
20                42.027203         135       0.030607               False   
21                21.346083         124       0.039424                True   
22                84.334328          82       0.026410                True   
23                71.138662         148       0.036468               False   
24                 8.614755         130       0.030786               False   
25                92.170002          95       0.033748                True   

            unit_cost_expansion  value_of_time  yield_curve  free_flow_time  \
experiment                                                                    
18                   140.210575       0.065729     0.000300              60   
19                    96.267985       0.077641     0.015375              60   
20                   107.239601       0.115429     0.004664              60   
21                   118.538455       0.014103     0.012946              60   
22                   133.676568       0.149598     0.017552              60   
23                   129.183633       0.056610     0.008356              60   
24                   123.140814       0.092145     0.000927              60   
25                   108.577268       0.128863     0.011068              60   

            initial_capacity  
experiment                    
18                       100  
19                       100  
20                       100  
21                       100  
22                       100  
23                       100  
24                       100  
25                       100

The demo module is set up to facilitate distributed multiprocessing. During the `setup`
step, the code detects if it is being run in a distributed "worker" environment instead of
in a normal Python environment.  If the "worker" environment is detected, then a copy
of the entire files-based model is made into the worker's local workspace, and the model
is run there instead of in the master workspace.  This allows each worker to edit the files
independently and simultaneously, without disturbing other parallel workers.

With this small modification, we are ready to run this demo model in parallel subprocesses.
to do, we simply import the `get_client` function, and use that for the `evaluator` argument
in the `run_experiments` method.

In [38]:
from emat.util.distributed import get_client # for multi-process operation
fx.run_experiments(design=design3, evaluator=get_client())

[00:40.81] MainProcess/INFO: performing 8 scenarios/policies * 1 model(s) = 8 experiments
[00:44.10] MainProcess/INFO: 1 cases completed
[00:44.14] MainProcess/INFO: 2 cases completed
[00:44.15] MainProcess/INFO: 3 cases completed
[00:44.16] MainProcess/INFO: 4 cases completed
[00:47.28] MainProcess/INFO: 5 cases completed
[00:47.31] MainProcess/INFO: 6 cases completed
[00:47.35] MainProcess/INFO: 7 cases completed
[00:47.36] MainProcess/INFO: 8 cases completed
[00:47.36] MainProcess/INFO: experiments finished


alpha      beta  input_flow  value_of_time  \
experiment                                                  
18          0.169616  4.064396         101       0.065729   
19          0.192963  4.973266         107       0.077641   
20          0.150234  5.429606         135       0.115429   
21          0.175508  4.505154         124       0.014103   
22          0.140599  5.035679          82       0.149598   
23          0.113669  3.831327         148       0.056610   
24          0.108685  4.452232         130       0.092145   
25          0.126817  3.599748          95       0.128863   

            unit_cost_expansion  interest_rate  yield_curve  expand_capacity  \
experiment                                                                     
18                   140.210575       0.034895     0.000300        26.090367   
19                    96.267985       0.027205     0.015375        56.044819   
20                   107.239601       0.030607     0.004664        42.027203   
21                   118.538455       0.039424     0.012946        21.346083   
22                   133.676568       0.026410     0.017552        84.334328   
23                   129.183633       0.036468     0.008356        71.138662   
24                   123.140814       0.030786     0.000927         8.614755   
25                   108.577268       0.033748     0.011068        92.170002   

            amortization_period debt_type  interest_rate_lock  free_flow_time  \
experiment                                                                      
18                           35     Paygo                True              60   
19                           46   GO Bond               False              60   
20                           31  Rev Bond               False              60   
21                           38  Rev Bond                True              60   
22                           21   GO Bond                True              60   
23                           25     Paygo               False              60   
24                           18   GO Bond               False              60   
25                           43  Rev Bond                True              60   

            initial_capacity  no_build_travel_time  build_travel_time  \
experiment                                                              
18                       100             70.596998          64.130206   
19                       100             76.209113          61.772878   
20                       100            105.981321          66.843277   
21                       100             87.754086          71.608550   
22                       100             63.105464          60.142764   
23                       100             90.628309          63.909220   
24                       100             80.971250          74.515731   
25                       100             66.326154          60.602481   

            time_savings  value_of_time_savings  net_benefits  \
experiment                                                      
18              6.466792              42.930410    -72.685387   
19             14.436236             119.930187   -123.276235   
20             39.138044             609.887232    370.661686   
21             16.145536              28.234915    -96.044488   
22              2.962700              36.343655   -689.465120   
23             26.719089             223.859479   -157.547999   
24              6.455519              77.329350      1.265260   
25              5.723673              70.069174   -405.972848   

            cost_of_capacity_expansion  present_cost_expansion  
experiment                                                      
18                          115.615797             3658.145376  
19                          243.206421             5395.321829  
20                          239.225545             4506.980463  
21                          124.279404             2530.331667  
22                  

## Running without a Database

It is possible to use the methods from TMIP-EMAT without connecting to a 
database at all, although this is not recommended.  The demo `RoadTestFileModel`
by default creates a database for you if one is not given, but it also 
allows explicitly disclaiming a database by setting the `db` argument to `False`.

In [39]:
fx_nodb = core_files_demo.RoadTestFileModel(db=False)

[00:47.39] MainProcess/WARNING: changing cwd to /var/folders/js/bk_dt9015j79_f6bxnc44dsr0000gp/T/tmpu0i09wc6


Without a database, the methods still work, but the data is not stored anywhere persistent.
So, for example, the `design_experiments` will return a pandas DataFrame containing a 
design, but not store it.

In [40]:
design4 = fx_nodb.design_experiments(design_name='lhs_4', n_samples=8, random_seed=4)
design4

alpha  amortization_period      beta debt_type  expand_capacity  \
0  0.157469                   43  5.352111   GO Bond        27.393080   
1  0.189268                   40  3.881977   GO Bond        56.222426   
2  0.140306                   50  4.234393   GO Bond        64.780568   
3  0.108730                   24  4.380424  Rev Bond        23.979788   
4  0.123790                   19  4.527048     Paygo         5.397776   
5  0.170412                   19  4.789556  Rev Bond        85.377351   
6  0.175064                   30  5.136301     Paygo        92.709704   
7  0.132154                   34  3.631101     Paygo        48.808345   

   input_flow  interest_rate  interest_rate_lock  unit_cost_expansion  \
0          86       0.035527               False            97.577951   
1         148       0.025682                True           141.368963   
2         112       0.037690                True           131.890137   
3          98       0.028841                True           126.122602   
4         115       0.038331               False           111.397305   
5         134       0.033783               False           114.269901   
6         126       0.027840                True           105.831242   
7          96       0.031697               False           136.741784   

   value_of_time  yield_curve  free_flow_time  initial_capacity  
0       0.125910     0.003238              60               100  
1       0.052622    -0.002455              60               100  
2       0.034445     0.013091              60               100  
3       0.078663     0.008845              60               100  
4       0.071495     0.002162              60               100  
5       0.090636     0.018692              60               100  
6       0.205397     0.007812              60               100  
7       0.112699     0.015616              60               100

If you try to run these experiments by giving the design name, it will fail,
because there is no database to query to convert that name into experimental
parameters.

In [41]:
try:
    fx_nodb.run_experiments('lhs_4')
except ValueError as error:
    log.error(repr(error))

[00:47.47] MainProcess/ERROR: ValueError('cannot load design "lhs_4", there is no db')


In [42]:
from emat.examples import road_test

In [43]:
_s, _db, _m = road_test()

[00:47.51] MainProcess/INFO: running script scope.sql
[00:47.51] MainProcess/INFO: running script exp_design.sql
[00:47.51] MainProcess/INFO: running script meta_model.sql


In [44]:
_m.run_experiments(design3, db=False)

[00:47.54] MainProcess/INFO: performing 8 scenarios/policies * 1 model(s) = 8 experiments
[00:47.55] MainProcess/INFO: performing experiments sequentially
[00:47.55] MainProcess/INFO: 1 cases completed
[00:47.56] MainProcess/INFO: 2 cases completed
[00:47.56] MainProcess/INFO: 3 cases completed
[00:47.56] MainProcess/INFO: 4 cases completed
[00:47.56] MainProcess/INFO: 5 cases completed
[00:47.56] MainProcess/INFO: 6 cases completed
[00:47.56] MainProcess/INFO: 7 cases completed
[00:47.56] MainProcess/INFO: 8 cases completed
[00:47.56] MainProcess/INFO: experiments finished


alpha      beta  input_flow  value_of_time  \
experiment                                                  
18          0.169616  4.064396         101       0.065729   
19          0.192963  4.973266         107       0.077641   
20          0.150234  5.429606         135       0.115429   
21          0.175508  4.505154         124       0.014103   
22          0.140599  5.035679          82       0.149598   
23          0.113669  3.831327         148       0.056610   
24          0.108685  4.452232         130       0.092145   
25          0.126817  3.599748          95       0.128863   

            unit_cost_expansion  interest_rate  yield_curve  expand_capacity  \
experiment                                                                     
18                   140.210575       0.034895     0.000300        26.090367   
19                    96.267985       0.027205     0.015375        56.044819   
20                   107.239601       0.030607     0.004664        42.027203   
21                   118.538455       0.039424     0.012946        21.346083   
22                   133.676568       0.026410     0.017552        84.334328   
23                   129.183633       0.036468     0.008356        71.138662   
24                   123.140814       0.030786     0.000927         8.614755   
25                   108.577268       0.033748     0.011068        92.170002   

            amortization_period debt_type  interest_rate_lock  free_flow_time  \
experiment                                                                      
18                           35     Paygo                True              60   
19                           46   GO Bond               False              60   
20                           31  Rev Bond               False              60   
21                           38  Rev Bond                True              60   
22                           21   GO Bond                True              60   
23                           25     Paygo               False              60   
24                           18   GO Bond               False              60   
25                           43  Rev Bond                True              60   

            initial_capacity  no_build_travel_time  build_travel_time  \
experiment                                                              
18                       100             70.596998          64.130206   
19                       100             76.209113          61.772878   
20                       100            105.981321          66.843277   
21                       100             87.754086          71.608550   
22                       100             63.105464          60.142764   
23                       100             90.628309          63.909220   
24                       100             80.971250          74.515731   
25                       100             66.326154          60.602481   

            time_savings  value_of_time_savings  net_benefits  \
experiment                                                      
18              6.466792              42.930410    -72.685387   
19             14.436236             119.930187   -131.395990   
20             39.138044             609.887232    376.276926   
21             16.145536              28.234915    -96.044488   
22              2.962700              36.343655   -689.465120   
23             26.719089             223.859479   -155.256907   
24              6.455519              77.329350      1.259730   
25              5.723673              70.069174   -405.972848   

            cost_of_capacity_expansion  present_cost_expansion  
experiment                                                      
18                          115.615797             3658.145376  
19                          251.326177             5395.321829  
20                          233.610306             4506.980463  
21                          124.279404             2530.331667  
22                  